In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline



# 读入数据

In [34]:
import stock
from stock.common.sdata import StockData
sd = StockData(code='600016')
data  = sd.combine_income(ndays=5)

# 计算N天的最大值

In [23]:
dd = sd.datas.tail(10)
Open,Close,Low,High = dd.Open,dd.Close,dd.Low,dd.High
shift_num = -1
Income = ((((Close.shift(shift_num).values + Low.shift(shift_num).values + High.shift(shift_num).values)/3)/Close) -1) * 100
ndays = 5
columns = []
for i in range(ndays):
    shift_num = 0 - (i+1)
    columns.append('IM_'+str(i+1))
    dd['IM_'+str(i+1)]= ((((Close.shift(shift_num).values + Low.shift(shift_num).values + High.shift(shift_num).values )/3)/Close) -1) * 100
dd = dd.dropna(axis=0,how='any') 
dd['INCOME'] = dd[columns].max(axis=1)
# for key in columns:
#     del dd[key]

/home/hecong/venv/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [24]:
print(dd)

            Open  High  Close   Low    Volume      Amount    INCOME  Flag  \
Date                                                                        
2018-05-21  7.94  7.98   7.94  7.93  55498235  44136736.0 -0.587741     0   
2018-05-22  7.95  7.96   7.88  7.84  65749035  51775721.6 -0.846024     0   
2018-05-23  7.87  7.87   7.79  7.78  61488114  48075734.4  0.299529     0   
2018-05-24  7.79  7.86   7.81  7.77  42686010  33378304.0 -0.085361     0   
2018-05-25  7.83  7.85   7.78  7.76  31007140  24187774.4  0.299914     0   

                IM_1      IM_2      IM_3      IM_4      IM_5  
Date                                                          
2018-05-21 -0.587741 -1.595298 -1.595298 -1.805206 -1.721243  
2018-05-22 -0.846024 -0.846024 -1.057530 -0.972927 -1.269036  
2018-05-23  0.299529  0.085580  0.171160 -0.128370 -1.626016  
2018-05-24 -0.170721 -0.085361 -0.384123 -1.877934 -1.835254  
2018-05-25  0.299914  0.000000 -1.499572 -1.456727 -1.071123  


# 基本统计信息

    https://blog.csdn.net/qq_39521554/article/details/79574561
    https://blog.csdn.net/cymy001/article/details/78300900

In [50]:
# 通过dataframe group by 显示
print('flag = 0', len(data.groupby('Flag').groups[0]))
print('flag = 1', len(data.groupby('Flag').groups[1]))
print('flag = 2', len(data.groupby('Flag').groups[2]))

flag = 0 291
flag = 1 1081
flag = 2 1484


# 数据归一化

    https://www.zhihu.com/question/20467170    标准化和归一化什么区别？
    
    http://www.cnblogs.com/zhaokui/p/5112287.html   归一化与标准化
    
    https://blog.csdn.net/zbc1090549839/article/details/44755451    时间序列的归一化方法
    
    https://machinelearningmastery.com/normalize-standardize-time-series-data-python/ How to Normalize and Standardize Time Series Data in Python
    
    
    归一化（Normalization）与标准化（Standardization）
    
    A. 归一化
    特点
        对不同特征维度的伸缩变换的目的是使各个特征维度对目标函数的影响权重是一致的，即使得那些扁平分布的数据伸缩变换成类圆形。这也就改变了原始数据的一个分布。归一化是一种简化计算的方式，即将有量纲的表达式，经过变换，化为无量纲的表达式，成为标量。 在多种计算中都经常用到这种方法。

    好处：
        1 提高迭代求解的收敛速度
        2 提高迭代求解的精度        
    
    B. 标准化
    特点
        对不同特征维度的伸缩变换的目的是使得不同度量之间的特征具有可比性。同时不改变原始数据的分布。数据的标准化（normalization）是将数据按比例缩放，使之落入一个小的特定区间。在某些比较和评价的指标处理中经常会用到，去除数据的单位限制，将其转化为无量纲的纯数值，便于不同单位或量级的指标能够进行比较和加权。

    好处
        1 使得不同度量之间的特征具有可比性，对目标函数的影响体现在几何分布上，而不是数值上
        2 不改变原始数据的分布
        
        
        
     


In [35]:
from sklearn.preprocessing import StandardScaler
data_in, target = StockData.package_data(data)
# scale = StandardScaler()
# data_scale = [scale.fit_transform(x.values) for x in data_in]

# 因数据每天都有新增变化， 所以暂时先不使用数据标准化、归一化处理，只是将数据做np.log


In [33]:
import numpy as np
data_scale = [np.log(x) for x in data_in]
print(data_scale)


[                Open      High     Close       Low     Volume     Amount
Date                                                                    
2006-08-02  1.403643  1.408545  1.393766  1.381282  17.486608  16.576562
2006-08-03  1.398717  1.410987  1.401183  1.391282  16.701846  15.800006
2006-08-04  1.401183  1.425515  1.393766  1.391282  17.393087  16.499905
2006-08-07  1.391282  1.393766  1.371181  1.368639  17.065797  16.148088
2006-08-09  1.396245  1.398717  1.386294  1.383791  16.780806  15.867448,                 Open      High     Close       Low     Volume     Amount
Date                                                                    
2006-08-03  1.398717  1.410987  1.401183  1.391282  16.701846  15.800006
2006-08-04  1.401183  1.425515  1.393766  1.391282  17.393087  16.499905
2006-08-07  1.391282  1.393766  1.371181  1.368639  17.065797  16.148088
2006-08-09  1.396245  1.398717  1.386294  1.383791  16.780806  15.867448
2006-08-10  1.386294  1.401183  1.398717  1.38379

# Pytorch DataLoader